# Develop and publish tools - API

A common pattern for developing tools is to connect RESTful APIs with LLM Agents. This pattern is used in the `xentropy-geocoding` tool where the Google Map API is used to perform geocoding.

In [1]:
from xentropy.client import Client

In [ ]:
client = Client(api_key='YOUR_API_KEY')

In [ ]:
from hashlib import sha256
import time
# XEntropy will send a Webhook-Secret header to your endpoint for authentication.
# You can replace YOUR_OWN_SEED with time.time() or other random string.
webhook_secret = sha256(b'YOUR_OWN_SEED').hexdigest()

# Replace with your own google cloud api key
env = {
    "WEBHOOK_SECRET": webhook_secret,
    "GOOGLE_CLOUD_API_KEY": "YOUR_GOOGLE_CLOUD_API_KEY",
}

with open('.env', 'w') as f:
    f.write('\n'.join([f'{key}={value}' for key, value in env.items()]))

In [ ]:
# Code snippet of wrapping an API into a tool to be published on XEntropy

from pydantic import BaseModel
import os
import requests

class Address(BaseModel):
    address: str


class Coordinate(BaseModel):
    latitude: float
    longitude: float


def geocoding(address: Address):

    geocoding = requests.get(
        'https://maps.googleapis.com/maps/api/geocode/json',
        params={
            'address': address.address,
            # YOUR Google Cloud API Key
            'key': os.environ.get('GOOGLE_CLOUD_API_KEY')
        }
    ).json()
    location = geocoding.get('results')[0].get('geometry').get('location')
    result = {'latitude': location.get(
        'lat'), 'longitude': location.get('lng')}

    return result

In [ ]:
# Test if the code snippet works
geocoding(address=Address(address='ENTER_YOUR_ADDRESS'))

Execute the following to start a FastAPI server on your virtual machine.
```bash
uvicorn server:app --host 0.0.0.0 --port 80 --reload
```

In [ ]:
import requests

# Test if the server is working
url = f'http://IP_OF_YOUR_VIRTUAL_MACHINE/geocoding'
response = requests.post(
    url,
    json={
        'address': 'JFK International Airport'
    },
    headers={
        'Webhook-Secret': webhook_secret
    }
)
response.json()

In [ ]:
from xentropy.tool import Tool

geocoding = Tool(
    api_key=client.api_key,
    name='geocoding', # name of the tool published, must be unique in your account
    description='Retrieve the latitude and longitude given an address using the highly accurate Google Map API.', # subject to prompt engineering optimisation
    endpoint=url,
    input_model=Address,
    output_model=Coordinate, # optional. If you have a output model defined it will aid user to write better code.
    price=1000, # xentropy_credit per request. 0 means free to use. 1 USD = 100,000 xentropy_credit
    free_quota=20,  # free use quota per user per day
)

tool_upload = geocoding.publish(
    webhook_secret=webhook_secret,
    # set to False if you want to test the tool before publishing it, or you want to keep it private.
    public=True
)

In [ ]:
# confirm that the tool works
geocoding = Tool.load(tool_upload['name'], api_key=client.api_key)
# you are not charged for using your own tool
geocoding.run(address='JFK International Airport')